In [3]:
import multiprocessing

multiprocessing.cpu_count()

12

In [5]:
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from sklearn.metrics import accuracy_score

# Check for CUDA
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Pneumonia Dataset Class
class PneumoniaDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for label in ['NORMAL', 'PNEUMONIA']:
            class_dir = os.path.join(self.root_dir, label)
            for img_name in os.listdir(class_dir):
                self.image_paths.append(os.path.join(class_dir, img_name))
                self.labels.append(0 if label == 'NORMAL' else 1)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Define Transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Datasets
train_dataset = PneumoniaDataset(root_dir='chest_xray/train', transform=transform)
test_dataset = PneumoniaDataset(root_dir='chest_xray/test', transform=transform)
val_dataset = PneumoniaDataset(root_dir='chest_xray/val', transform=transform)

# Create Data Loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Load Pretrained Model
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 2)  # NORMAL, PNEUMONIA
model = model.to(device)

# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 8
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')

    # Validation Accuracy
    model.eval()
    val_labels, val_preds = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            val_labels.extend(labels.cpu().numpy())
            val_preds.extend(preds.cpu().numpy())

    val_accuracy = accuracy_score(val_labels, val_preds)
    print(f'Validation Accuracy: {val_accuracy:.4f}')

# Test Accuracy
model.eval()
test_labels, test_preds = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        test_labels.extend(labels.cpu().numpy())
        test_preds.extend(preds.cpu().numpy())

test_accuracy = accuracy_score(test_labels, test_preds)
print(f'Test Accuracy: {test_accuracy:.4f}')

# Save Model
torch.save(model.state_dict(), 'pneumonia_classified.pth')
print("Model saved successfully!")


Epoch 1/8, Loss: 0.1301105636103433
Validation Accuracy: 0.5625
Epoch 2/8, Loss: 0.06039675644118956
Validation Accuracy: 0.7500
Epoch 3/8, Loss: 0.047584456623213346
Validation Accuracy: 1.0000
Epoch 4/8, Loss: 0.041832494119859294
Validation Accuracy: 1.0000
Epoch 5/8, Loss: 0.03417607397305282
Validation Accuracy: 0.9375
Epoch 6/8, Loss: 0.0239329907288857
Validation Accuracy: 0.8125
Epoch 7/8, Loss: 0.0311543543668081
Validation Accuracy: 1.0000
Epoch 8/8, Loss: 0.008236925409624868
Validation Accuracy: 1.0000
Test Accuracy: 0.8093
Model saved successfully!
